# 분석목표

1. 구매 소요 시간 분석
    - 1-1. 가입일로부터 구매일까지 전환된 소요시간 분석
         - 최근 3개월 전체 분석 / 각 월 별 분석
         - 분기 경향 / 월 별 변동사항 확인 -> 트렌드 분석
             - ex. 가입 1일 이내 전환 유저 : 10% -> 15% 상승 성과
             
2. 구매 상품 트렌드 분석
    - 2-1. 유저당 주문 수량 분석
         - 최근 3개월 전체 분석 / 각 월 별 분석
         - 분기 경향 / 월 별 변동사항 확인 -> 트렌드 분석

    - 2-2. 주문 아이템 분석
         - 브랜드 별 선호도, 아이템 카테고리별 선호도 분석
         - 신규 유저들의 구매 트렌드 파악
         
         
3. 분석 내용 토대로 인사이트 도출
    - 3-1. 분석 내용 도식화
         - 1~2의 분석 내용 종합 및 도식화 (표 또는 그래프)
         - 유사성을 갖는 결과에 대하여 그룹화
    - 3-2. 인사이트 도출
         - 해당 결과를 통해 나타나는 사항 정리
         - 인사이트 도출 혹은 새로운 제안사항 전달 (없을 시 진행하지 않아도 무방)


용어 및 칼럼 설명
- brandnm : 아이템 브랜드명
- goodsnm : 아이템 명
- USER : 개별 유저 식별 값
- reg_date : 개별 유저 가입일
- order_date : 아이템 주문일
- p_type : 주문 상품 종류
     - 7days (7일권) / 4days (4일권)
- sub_type : 주문 상품 카테고리 
     - BA – 가방 / O – 아우터 / D – 드레스 / T – 상의 / B – 하의 / TB – 투피스 
- app_yn : 어플리케이션 사용 여부
     - Y – 어플리케이션 사용 / (공백) – 어플리케이션 미사용


# 생각정리

결국 목표는 2개
   - 구매까지 걸리는 시간 분석 (각 월별 / 분기별 / // 트렌트 분석)
   - 유저당 구매 분석

In [4]:
import os 
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'  # (Windows 용) 한글 출력을 위한 글꼴 설정

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

# 랜덤시드 통일
np.random.seed(42)

In [5]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux
        # colab에서는 runtime을 <꼭> 재시작 해야함.
        # 런타임을 재시작 하지 않고 폰트 설치를 하면 기본 설정 폰트가 로드되어 한글이 깨짐.
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null

        import matplotlib.font_manager as fm

        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

In [6]:
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당.
a = get_font_family()
# 폰트설정
import matplotlib.pyplot as plt 
plt.rc("font", family = a)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# ggplot으로 그래프 스타일 설정 / 개인 자유
plt.style.use("ggplot")

In [7]:
# 데이터 확인
os.listdir('data')

['클로젯셰어_과제DB 대체 자료.xlsx', '클로젯셰어_수행 과제.docx']

In [8]:
# 데이터프레임 불러오기
df = pd.read_excel("./data/클로젯셰어_과제DB 대체 자료.xlsx")

In [11]:
# 데이터 내부 살펴보기 함수
def search_data(df):
    print("데이터 확인 :")
    display(df.head())
    print()
    print()
    
    print("데이터 살펴보기 :")
    display(df.describe())
    print()
    print()    
    
    print("데이터 정보 확인 :")
    print(df.info())
    print()
    print()
        
    print("데이터 형태 확인 :", df.shape)
    print()
    print()
    
    print("결측치 확인 :")
    print(df.isna().sum())
    print()
    print()
    
    print("유니크값 확인 :")
    for col in df.columns:
        print(f"column : {col}")
        print(f"The number of unique : {df[col].nunique()}")
        print()

In [12]:
search_data(df)

데이터 확인 :


,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
0,Gucci,flower dionysus shoulder bag beige,a161237,2019-12-06,2021-01-15,7days,BA,Y
1,El estilo de Claire,beads cropped jacket black,a1140859,2021-01-02,2021-01-15,4days,O,NaN
2,Danha,embroidery point oriental mini skirt blue,a1140859,2021-01-02,2021-01-15,4days,B,NaN
3,Danha,graphics pattern top pink,a1140859,2021-01-02,2021-01-15,4days,T,NaN
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,2021-01-01,2021-01-15,7days,O,Y




데이터 살펴보기 :


,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
count,1442,1471,1471,1466,1471,1471,1471,1295
unique,245,979,1234,368,90,2,6,1
top,Gucci,chevron quilted cross bag black,a1149461,2021-03-16 00:00:00,2021-03-24 00:00:00,4days,BA,Y
freq,232,23,11,44,47,894,692,1295
first,NaN,NaN,NaN,2017-06-24 00:00:00,2021-01-01 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,2021-03-31 00:00:00,2021-03-31 00:00:00,NaN,NaN,NaN




데이터 정보 확인 :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   brandnm     1442 non-null   object        
 1   goodsnm     1471 non-null   object        
 2   USER        1471 non-null   object        
 3   reg_date    1466 non-null   datetime64[ns]
 4   order_date  1471 non-null   datetime64[ns]
 5   p_type      1471 non-null   object        
 6   sub_type    1471 non-null   object        
 7   app_yn      1295 non-null   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 92.1+ KB
None


데이터 형태 확인 : (1471, 8)


결측치 확인 :
brandnm        29
goodsnm         0
USER            0
reg_date        5
order_date      0
p_type          0
sub_type        0
app_yn        176
dtype: int64


유니크값 확인 :
column : brandnm
The number of unique : 245

column : goodsnm
The number of unique : 979

column : USER
The number of unique : 1234